In [1]:
import os
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_core.tools import Tool
from langchain.chains import LLMMathChain

In [2]:
key_file='../openai_api_key.txt'
with open(key_file, 'r') as file:
    openai_api_key=file.read()
os.environ["OPENAI_API_KEY"] = openai_api_key

In [3]:
verbose=False

# This simple prompt simulates the case of accessing private information.
concealedinfo_prompt = PromptTemplate.from_template("Mary is 24. Given this information, answer the following question: {question}")
concealedinfo_chain = LLMChain(llm=OpenAI(temperature=0), prompt=concealedinfo_prompt, verbose=verbose)

api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=100)
wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper)

calculate_chain = LLMMathChain.from_llm(llm=OpenAI(temperature=0), verbose=verbose) 

tools=[ wiki_tool,
        Tool(
            name="CALCULATE",
            func=calculate_chain.run, 
            description="Useful for making math calculations. Input: a calculation to perform on a set of numbers (these need to be numbers, not literal expressions). Outputs: the result of the calculation."
        ),
        Tool(
            name="INFO_ABOUT_MARY",
            func=concealedinfo_chain.run,
            description='Only needed to answer a question that requires knowledge about Mary. Do not use unless Mary is involved.' 
        ),
]


In [4]:

prompt = hub.pull("hwchase17/openai-functions-agent") # If you want to see the prompt in full, you can at: https://smith.langchain.com/hub/hwchase17/openai-functions-agent

llm = ChatOpenAI(temperature=0.1)

agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=verbose, return_intermediate_steps=True
)

In [9]:
for i in range(5):
    response = agent_executor.invoke({"input": "What is the sum of the ages of Mary and of Leonardo Di Caprio (as of 2023)?"})
    answer=response['output']
    print(f"{answer=}")
    if '73' not in answer:
        print(response['intermediate_steps'])

answer="Leonardo DiCaprio was born on November 11, 1974. Let's calculate the sum of the ages of Mary and Leonardo DiCaprio as of 2023.\n\nMary's age: 24\nLeonardo DiCaprio's age in 2023: 2023 - 1974 = 49\n\nSum of their ages: 24 + 49 = 73\n\nTherefore, the sum of the ages of Mary and Leonardo DiCaprio as of 2023 is 73."
answer="Leonardo DiCaprio was born on November 11, 1974. Let's calculate the sum of the ages of Mary and Leonardo DiCaprio as of 2023.\n\nMary's age: 24\nLeonardo DiCaprio's age in 2023: 2023 - 1974 = 49\n\nSum of their ages: 24 + 49 = 73\n\nTherefore, the sum of the ages of Mary and Leonardo DiCaprio as of 2023 is 73."
answer="Leonardo DiCaprio was born on November 11, 1974. Let's calculate the sum of the ages of Mary and Leonardo DiCaprio as of 2023.\n\nMary's age in 2023: 24\nLeonardo DiCaprio's age in 2023: 2023 - 1974 = 49\n\nSum of their ages: 24 + 49 = 73\n\nTherefore, the sum of the ages of Mary and Leonardo DiCaprio as of 2023 is 73."
answer='The sum of the age

We expect the answer to be 71, as the LLM will use 2021 as reference year. Any answer that does not contain 71 will make the code print the intermediate steps for inspection.